In [1]:
import torch

import random
import numpy as np
import onnx
import onnx_tf

def manual_seed(seed):
    np.random.seed(seed) #1
    random.seed(seed) #2
    torch.manual_seed(seed) #3
    torch.cuda.manual_seed(seed) #4.1
    torch.cuda.manual_seed_all(seed) #4.2
    torch.backends.cudnn.benchmark = False #5 
    torch.backends.cudnn.deterministic = True #6

manual_seed(42)

In [2]:
filepath = './checkpoint/vgg16.pth'
checkpoint = torch.load(filepath)
# torch_model = vgg.load_vgg16(num_classes= 100)
torch_model = checkpoint['model']
print(torch_model)
torch_model.load_state_dict(checkpoint['model_state_dict'])

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

<All keys matched successfully>

In [3]:
torch_model.eval()
input = torch.randn(1,3,224,224)
print(input.shape, input.dtype)
output = torch_model(input)
print(output.shape)
onnx_path = "./checkpoint/vgg16.onnx"

torch.onnx.export(model= torch_model,               # 실행될 모델
                  args=input,                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                  f= onnx_path,   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                  export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부
                  verbose=True,
                  input_names = ['inputs'],   # 모델의 입력값을 가리키는 이름
                  output_names = ['outputs'], # 모델의 출력값을 가리키는 이름
                  opset_version = 15,
                  do_constant_folding=True,  # 최적화시 상수폴딩을 사용할지의 여부
)

torch.Size([1, 3, 224, 224]) torch.float32
torch.Size([1, 512, 7, 7])
avgpool -> torch.Size([1, 512, 7, 7])
torch.Size([1, 100])
torch.Size([1, 512, 7, 7])
avgpool -> torch.Size([1, 512, 7, 7])
Exported graph: graph(%inputs : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %features.0.weight : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %features.0.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %features.2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.5.weight : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %features.5.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %features.7.weight : Float(128, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %features.7.bias : Float(128, strides=[1],

In [ ]:
# load onnx model
onnx_model = onnx.load(onnx_path)

# convet onnx model to tensorflow
tf_model = onnx_tf.backend.prepare(onnx_model)
tf_model_path = "./checkpoint/vgg16.pb"
tf_model.export_graph(tf_model_path)